# Custom Staged PassManager

In [3]:
from qiskit.transpiler.passmanager import StagedPassManager
from qiskit.transpiler.passmanager_config import PassManagerConfig
from qiskit.transpiler.preset_passmanagers.plugin import PassManagerStagePluginManager

In [5]:
def qopt_pass_manager(pass_manager_config: PassManagerConfig) -> StagedPassManager:
    """
    Applies user generated custom layout.
    Unused physical qubit is allocated to ancilla space.

    The pass manager unrolls the circuit to the desired basis and transforms the circuit to match the coupling map.

    
    """
    basis_gates = pass_manager_config.basis_gates
    coupling_map = pass_manager_config.coupling_map
    initial_layout = pass_manager_config.initial_layout
    init_method = pass_manager_config.init_method or "default"
    layout_method = pass_manager_config.layout_method or "default"
    routing_method = pass_manager_config.routing_method or "default"
    translation_method = pass_manager_config.translation_method or "default"
    optimization_method = pass_manager_config.optimization_method or "default"
    scheduling_method = pass_manager_config.scheduling_method or "default"
    target = pass_manager_config.target

    # Build pass manager
    plugin_manager= PassManagerStagePluginManager()

    routing_pm = plugin_manager.get_passmanager_stage(
        "routing", routing_method, pass_manager_config, optimization_level=0
    )

    if coupling_map or initial_layout:
        layout - plugin_manager.get_passmanager_stage(
            "layout", layout_method, pass_manager_config, optimization_level=0
        )
        routing = routing_pm
    else:
        layout = None
        routing = None

    translation = plugin_manager.get_passmanager_stage(
        "translation", translation_method, pass_manager_config, optimization_level=0
    )

    sched = plugin_manager.get_passmanager_stage(
        "scheduling", scheduling_method, pass_manager_config, optimization_level=0
    )

    pre_init = common.generate_control_flow_options_check(
        layout_method=layout_method,
        routing_method=routing_method,
        translation_method=translation_method,
        optimization_method=optimization_method,
        scheduling_method=scheduling_method,
        basis_gates=basis_gates,
        target=target,
    )

    init = plugin_manager.get_passmanager_stage(
        "init", init_method, pass_manager_config, optimization_level=0
    )
    optimization = plugin_manager.get_passmanager_stage(
        "optimization", optimization_method, pass_manager_config, optimization_level=0
    )
    
    return StagedPassManager(
        pre_init=pre_init,
        init=init,
        layout=custom_layout,
        routing=custom_routing,
        translation=translation,
        optimization=optimization,
        scheduling=sched
    )

## Generate custom layout

In [6]:
from qiskit.transpiler.layout import Layout

In [ ]:
class QopsLayout(Layout):
    """
    Generates a customized layout based on Christofides algorithm
    """
    def __init__(self, input_dict=None):
        